# Chapter 10: Introduction to Artificial Neural Networks with Keras

Bab ini memulai bagian kedua dari buku ini, yang berfokus pada Jaringan Saraf Tiruan (Artificial Neural Networks - ANNs) dan Deep Learning. Kita akan beralih dari Scikit-Learn ke **TensorFlow** dan API tingkat tingginya, **Keras**.

Kita akan membahas:
* Konsep dasar ANN, mulai dari inspirasi biologisnya hingga arsitektur modern seperti **Multilayer Perceptron (MLP)**.
* Bagaimana **backpropagation** bekerja untuk melatih ANN.
* Implementasi ANN menggunakan Keras untuk tugas klasifikasi dan regresi.
* Tiga API utama Keras: **Sequential API**, **Functional API**, dan **Subclassing API**.
* Cara menyimpan dan memulihkan model, menggunakan *callbacks*, dan memvisualisasikan pelatihan dengan **TensorBoard**.

## From Biological to Artificial Neurons

ANN terinspirasi oleh arsitektur otak. Model paling awal adalah **Perceptron**, yang diperkenalkan oleh Frank Rosenblatt pada tahun 1957. Perceptron adalah arsitektur ANN paling sederhana, berdasarkan unit komputasi yang disebut *threshold logic unit (TLU)*.

Sebuah TLU menghitung jumlah berbobot dari inputnya, lalu menerapkan *step function* untuk menghasilkan output. Sebuah Perceptron terdiri dari satu lapisan TLU. Namun, Perceptron memiliki kelemahan serius: mereka tidak mampu memecahkan beberapa masalah sepele (seperti masalah klasifikasi XOR). Keterbatasan ini dapat diatasi dengan menumpuk beberapa Perceptron, yang menghasilkan **Multilayer Perceptron (MLP)**.

### Multilayer Perceptron and Backpropagation

MLP terdiri dari satu *input layer*, satu atau lebih *hidden layers*, dan satu *output layer*. Ketika sebuah ANN memiliki tumpukan *hidden layers* yang dalam, ia disebut **Deep Neural Network (DNN)**.

MLP dilatih menggunakan algoritma **backpropagation**. Secara singkat, algoritma ini bekerja sebagai berikut:
1. Setiap *mini-batch* data dilewatkan melalui jaringan (*forward pass*).
2. Kesalahan (*error*) output jaringan diukur menggunakan *loss function*.
3. Kontribusi setiap koneksi terhadap *error* dihitung dengan bergerak mundur melalui jaringan (*reverse pass*) menggunakan *chain rule* dari kalkulus.
4. Bobot koneksi disesuaikan untuk mengurangi *error* menggunakan Gradient Descent.

## Implementing MLPs with Keras

Keras adalah API tingkat tinggi untuk membangun, melatih, dan mengevaluasi jaringan saraf. Kita akan menggunakan `tf.keras`, implementasi Keras yang dibundel dengan TensorFlow.

### Membangun Image Classifier menggunakan Sequential API
**Sequential API** adalah cara paling sederhana untuk membuat model di Keras, cocok untuk jaringan yang hanya merupakan tumpukan lapisan tunggal.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Memuat dataset Fashion MNIST
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

# Membagi dan melakukan penskalaan data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Membuat model menggunakan Sequential API
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Melihat ringkasan model
model.summary()

In [ ]:
# Mengkompilasi model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

# Melatih model
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

### Membangun Model Kompleks menggunakan Functional API

Untuk jaringan dengan topologi yang lebih kompleks, seperti yang memiliki banyak input atau output, kita dapat menggunakan **Functional API**. Ini memungkinkan kita untuk mendefinisikan grafik lapisan yang lebih rumit.

Contohnya, kita bisa membuat arsitektur **Wide & Deep** yang menghubungkan beberapa input langsung ke *output layer*.

In [ ]:
# Contoh Wide & Deep dengan multiple inputs
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)

model = keras.Model(inputs=[input_A, input_B], outputs=[output])

## Menyimpan dan Memulihkan Model

Keras memudahkan penyimpanan model yang telah dilatih ke dalam satu file. File ini akan berisi arsitektur model, bobot, konfigurasi pelatihan (*loss*, *optimizer*), dan status *optimizer*, memungkinkan Anda untuk melanjutkan pelatihan tepat di tempat Anda berhenti.

Untuk mencegah kehilangan progres pada pelatihan yang lama, kita dapat menggunakan *callbacks* seperti `ModelCheckpoint` untuk menyimpan *checkpoint* secara berkala, dan `EarlyStopping` untuk menghentikan pelatihan jika tidak ada kemajuan.

In [ ]:
# model.save("my_keras_model.h5")
# model = keras.models.load_model("my_keras_model.h5")

# Contoh penggunaan callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# history = model.fit(X_train, y_train, epochs=100,
#                     validation_data=(X_valid, y_valid),
#                     callbacks=[checkpoint_cb, early_stopping_cb])

## Fine-Tuning Hyperparameters

Fleksibilitas jaringan saraf membawa banyak *hyperparameter* untuk disesuaikan. Kita bisa menggunakan `GridSearchCV` atau `RandomizedSearchCV` dari Scikit-Learn untuk menemukan kombinasi *hyperparameter* terbaik dengan membungkus model Keras kita.

Beberapa *hyperparameter* kunci untuk disesuaikan:
* **Jumlah *hidden layers***
* **Jumlah neuron per *layer***
* ***Learning rate***
* ***Optimizer***
* ***Batch size***
* ***Activation function***